In [40]:
import pandas as pd
import numpy as np
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdb_api_results_2010.json',
 'final_tmdb_data_2018.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'tmdb_api_results_2011.json',
 'tmdb_api_results_2016.json',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2010.csv.gz',
 'title_basics.csv.gz',
 'tmdb_api_results_2017.json',
 'tmdb_api_results_2018.json',
 'final_tmdb_data_2019.csv.gz',
 'tmdb_api_results_2014.json',
 'final_tmdb_data_2015.csv.gz',
 'tmdb_api_results_2015.json',
 'final_tmdb_data_2017.csv.gz',
 'tmdb_api_results_2019.json',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'final_tmdb_data_2013.csv.gz',
 'tmdb_api_results_2012.json',
 'title_akas.csv.gz',
 'titleR.csv.gz',
 'tmdb_api_results_2013.json',
 'final_tmdb_data_2011.csv.gz']

In [41]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file(filename),
    Adapted from://www.geeksforgeeks.org/append-to-json-file-using-python/"""
    with open(filename,'r+') as file:
        #first we load existing file into a dict
        file_data = json.load(file)
        ##choose extend or append
        if (type(new_data) == list & type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # sets files current position at offset.
        file.seek(0)
        #convert back to json
        json.dump(file_data, file)

In [42]:
basics= pd.read_csv('Data/title_basics.csv.gz')

In [43]:
YEARS_TO_GET = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

In [44]:
errors = [ ]

In [45]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
# Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: create it
    if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
#Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
    movie_ids = df['tconst'].copy()
# Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
# filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
        final_year_df= pd.read_json(JSON_FILE)
        final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                            compression="gzip", index=False)

YEARS:   0%|          | 0/10 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/3827 [00:00<?, ?it/s]

Movies from 2011:   0%|          | 0/4196 [00:00<?, ?it/s]

Movies from 2012:   0%|          | 0/4489 [00:00<?, ?it/s]

Movies from 2013:   0%|          | 0/4677 [00:00<?, ?it/s]

...

Movies from 2015:   0%|          | 0/5011 [00:00<?, ?it/s]

Movies from 2016:   0%|          | 0/5214 [00:00<?, ?it/s]

Movies from 2017:   0%|          | 0/5583 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/5710 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/5815 [00:00<?, ?it/s]

In [49]:
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
6316,tt0385887,movie,Motherless Brooklyn,Motherless Brooklyn,0,2019.0,NaN,144,"Crime,Drama,Mystery"
8335,tt0437086,movie,Alita: Battle Angel,Alita: Battle Angel,0,2019.0,NaN,122,"Action,Adventure,Sci-Fi"
8511,tt0441881,movie,Danger Close,Danger Close: The Battle of Long Tan,0,2019.0,NaN,118,"Action,Drama,War"
8755,tt0448115,movie,Shazam!,Shazam!,0,2019.0,NaN,132,"Action,Adventure,Comedy"
11367,tt0783640,movie,The Last Full Measure,The Last Full Measure,0,2019.0,NaN,116,"Drama,War"


In [28]:
df_2010 = pd.read_csv('Data/final_tmdb_data_2010.csv.gz')
df_2011 = pd.read_csv('Data/final_tmdb_data_2011.csv.gz')
df = pd.concat([df_2010,df_2011] )
df.head()

,imdb_id
0,0
0,0


NameError: name 'df_2012' is not defined

In [20]:
df.value_counts('certification')

certification
R          455
PG-13      183
NR          68
PG          63
G           24
NC-17        6
-            1
Unrated      1
dtype: int64

In [37]:
Rrating = df.loc[df['certification']=='R']

KeyError: 'certification'

In [38]:
PG13rating = df.loc[df['certification']=='PG-13', '']

KeyError: 'certification'

In [39]:
PGrating= df.loc[df['certification']=='PG']

KeyError: 'certification'

In [16]:
Grating= df.loc[df['certification']=='G']

In [17]:
NC17rating= df.loc[df['certification']=='NC-17']